In [1]:
#import dask.dataframe as dd
import pandas as pd
import numpy as np
import dask.dataframe as dd
# Set up Dask to use multiple cores
from dask.distributed import Client,LocalCluster

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


import plotly.graph_objects as go
import plotly.io as pio
# Set default renderer
pio.renderers.default = 'iframe'  #  See [1]

# Set up the local cluster
cluster = LocalCluster(
    n_workers=10,  # Use one worker per core
    threads_per_worker=1,  # Single-threaded workers for CPU-bound tasks
    memory_limit='20GB'  # Allocate memory per worker
)

# Create the client
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 10
Total threads: 10,Total memory: 186.26 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41965,Workers: 10
Dashboard: http://127.0.0.1:8787/status,Total threads: 10
Started: Just now,Total memory: 186.26 GiB
Comm: tcp://127.0.0.1:46439,Total threads: 1
Dashboard: http://127.0.0.1:41197/status,Memory: 18.63 GiB
Nanny: tcp://127.0.0.1:43509,


# PA matrix

## Sample statistics

In [ ]:
import dask.dataframe as dd
import pandas as pd

# Set up the client as shown above

# Read the CSV file
df = dd.read_csv("../unitig_matrix_pa/unitigs.filter.02.csv", 
                 header=0, 
                 assume_missing=True, 
                 dtype='float32',
                 blocksize="1GB")  # Increased blocksize due to large memory

# Set the first column as the index if needed
df = df.set_index(df.columns[0])

# Define custom function for aggregation
def custom_agg(series):
    return pd.Series({
        'count': series.count(),
        'mean': series.mean(),
        'std': series.std(),
        'min': series.min(),
        'max': series.max(),
        '25%': series.quantile(0.25),
        '50%': series.quantile(0.50),
        '75%': series.quantile(0.75)
    })

# Apply the custom aggregation function
row_stats = df.apply(custom_agg, axis=1, meta={
    'count': 'float32',
    'mean': 'float32',
    'std': 'float32',
    'min': 'float32',
    'max': 'float32',
    '25%': 'float32',
    '50%': 'float32',
    '75%': 'float32'
})

# Compute the results
print("Computing statistics...")
combined_stats = row_stats.compute()

# Save the results
print("Saving results...")
combined_stats.to_csv('pa_unitig_matrix_02_filtered_row_statistics.csv')
print("Done!")

# Close the client
client.close()

## Unitig statistics

In [ ]:
import dask.dataframe as dd
import pandas as pd

# Set up the client as shown above

# Read the CSV file
df = dd.read_csv("../unitig_matrix_pa/unitigs.filter.02.csv", 
                 header=0, 
                 assume_missing=True, 
                 dtype='float32',
                 blocksize="1GB")  # Increased blocksize due to large memory

# Set the first column as the index if needed
df = df.set_index(df.columns[0])

# Define custom function for aggregation
def custom_agg(series):
    return pd.Series({
        'count': series.count(),
        'mean': series.mean(),
        'std': series.std(),
        'min': series.min(),
        'max': series.max(),
        '25%': series.quantile(0.25),
        '50%': series.quantile(0.50),
        '75%': series.quantile(0.75)
    })

# Apply the custom aggregation function
row_stats = df.apply(custom_agg, axis=1, meta={
    'count': 'float32',
    'mean': 'float32',
    'std': 'float32',
    'min': 'float32',
    'max': 'float32',
    '25%': 'float32',
    '50%': 'float32',
    '75%': 'float32'
})

# Compute the results
print("Computing statistics...")
combined_stats = row_stats.compute()

# Save the results
print("Saving results...")
combined_stats.to_csv('pa_unitig_matrix_02_filtered_row_statistics.csv')
print("Done!")

# Close the client
client.close()

## Hypothesis tests

In [ ]:
metadata=pd.read_csv("../metadata/Damselflies_metadata_with_morph.csv",index_col=0)
# Load column names from fof_filtered_pa.txt
with open('../metadata/fof_filtered_pa.txt', 'r') as f:
    column_names = [line.strip() for line in f]
column_names = [name.replace("data/unitigs/", "").replace(".unitigs.fa.gz", "") for name in column_names]


In [ ]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.multitest import multipletests

# Create a dictionary mapping sample names to their morphs
sample_to_morph = dict(zip(metadata['Run'], metadata['Morph']))  # adjust 'SampleID' if it's named differently

# Create dictionaries for each morph group
morph_groups = {morph: [] for morph in metadata['Morph'].unique()}

for sample in column_names:
    if sample in sample_to_morph:
        morph = sample_to_morph[sample]
        morph_groups[morph].append(sample)

# Print group sizes for verification
for morph, samples in morph_groups.items():
    print(f"Group {morph}: {len(samples)} samples")

# Load your data, specifying the column names
df = dd.read_csv('../unitig_matrix_pa/unitigs.filter.02.csv', header=None, names=['unitig_id'] + column_names)

def epsilon_squared(h_statistic, n):
    return h_statistic / (n**2 - 1) / (n + 1)

def compute_test(row):
    group_data = {morph: row[samples].values for morph, samples in morph_groups.items()}
    
    # Check if all groups have variation
    if all(len(set(group)) > 1 for group in group_data.values()):
        # Perform Kruskal-Wallis H-test
        h_statistic, p_value = stats.kruskal(*group_data.values())
        n = sum(len(group) for group in group_data.values())
        effect_size = epsilon_squared(h_statistic, n)
    else:
        # If any group has no variation, set h_statistic and effect_size to 0 and p_value to 1
        h_statistic, p_value, effect_size = 0, 1, 0
    
    return pd.Series({'h_statistic': h_statistic, 'p_value': p_value, 'effect_size': effect_size})

# Define the metadata for Dask
meta = pd.DataFrame(columns=['h_statistic', 'p_value', 'effect_size']).astype({'h_statistic': 'float64', 'p_value': 'float64', 'effect_size': 'float64'})

# Apply the test to each chunk
results = df.map_partitions(lambda chunk: chunk.apply(compute_test, axis=1), meta=meta).compute()

# Perform Benjamini-Hochberg correction
_, adjusted_p_values, _, _ = multipletests(results['p_value'], method='fdr_bh')

# Add adjusted p-values to the results
results['adjusted_p_value'] = adjusted_p_values

# Calculate the number of significant results
alpha = 0.05  # Set your desired significance level
significant_count = sum(results['adjusted_p_value'] < alpha)

print(f"Number of tests performed: {len(results)}")
print(f"Number of significant results at α = {alpha}: {significant_count}")
print(f"Proportion of significant results: {significant_count / len(results):.2%}")

# Optional: Save results to file
results.to_csv('kruskal_wallis_results_pa_matrix.csv', index=False)

# Display summary statistics
print("\nSummary Statistics:")
print(results.describe())

# Optional: Display top significant results
print("\nTop 10 significant unitigs:")
print(results.sort_values('adjusted_p_value').head(10))

In [ ]:
results.shape

# Abundance matrix

## Sample statistics

In [ ]:
import dask.dataframe as dd

#client = Client(n_workers=20, memory_limit='40GB')

df = dd.read_csv("../unitig_matrix_abundance/A.mat",sep=" ")

# Set the first column as the index
df = df.set_index(df.columns[0])

# Calculate statistics for each column
stats = df.describe().compute()

# Calculate additional quantiles
quantiles = df.quantile([0.25, 0.75]).compute()

# Combine the statistics and quantiles
combined_stats = stats.transpose()
combined_stats['25%'] = quantiles.loc[0.25]
combined_stats['75%'] = quantiles.loc[0.75]

# Reorder columns
column_order = ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']
combined_stats = combined_stats[column_order]

# Print the combined statistics
print(combined_stats)

# Save the results to a CSV file
combined_stats.to_csv('abundance_unitig_matrix_02_filtered_col_statistics.csv')


## Unitig statistics

In [ ]:
import dask.dataframe as dd
import pandas as pd

# Set up the client as shown above

# Read the CSV file
df = dd.read_csv("../unitig_matrix_abundance/A.mat", 
                 sep='\s+',
                 header=0, 
                 assume_missing=True, 
                 dtype='float32',
                 blocksize="100MB")  # Adjust this value as needed

# Set the first column as the index if needed
df = df.set_index(df.columns[0])

# Define custom function for aggregation
def custom_agg(series):
    return pd.Series({
        'count': series.count(),
        'mean': series.mean(),
        'std': series.std(),
        'min': series.min(),
        'max': series.max(),
        '25%': series.quantile(0.25),
        '50%': series.quantile(0.50),
        '75%': series.quantile(0.75)
    })

# Apply the custom aggregation function
row_stats = df.apply(custom_agg, axis=1, meta={
    'count': 'float32',
    'mean': 'float32',
    'std': 'float32',
    'min': 'float32',
    'max': 'float32',
    '25%': 'float32',
    '50%': 'float32',
    '75%': 'float32'
})

# Compute the results
print("Computing statistics...")
combined_stats = row_stats.compute()

# Save the results
print("Saving results...")
combined_stats.to_csv('abundance_unitig_matrix_02_filtered_row_statistics.csv')
print("Done!")

# Close the client
client.close()

In [ ]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.multitest import multipletests
# Create a dictionary mapping sample names to their morphs
sample_to_morph = dict(zip(metadata['Run'], metadata['Morph']))  # adjust 'Run' if it's named differently

# Create dictionaries for each morph group
morph_groups = {morph: [] for morph in metadata['Morph'].unique()}

for sample in column_names:
    if sample in sample_to_morph:
        morph = sample_to_morph[sample]
        morph_groups[morph].append(sample)

# Print group sizes for verification
for morph, samples in morph_groups.items():
    print(f"Group {morph}: {len(samples)} samples")

# Load your data, specifying the column names
df = dd.read_csv("../unitig_matrix_abundance/A.mat", 
                 sep='\s+',
                 assume_missing=True, 
                 dtype='float32',
                 blocksize="100MB",  # Adjust this value as needed
                 header=None, 
                 names=['unitig_id'] + column_names)

def epsilon_squared(h_statistic, n):
    return h_statistic / ((n**2 - 1) / (n + 1))

def compute_test(row):
    group_data = {morph: row[samples].values for morph, samples in morph_groups.items()}
    
    if all(len(set(group)) > 1 for group in group_data.values()):
        h_statistic, p_value = stats.kruskal(*group_data.values())
        n = sum(len(group) for group in group_data.values())
        effect_size = epsilon_squared(h_statistic, n)
    else:
        h_statistic, p_value, effect_size = 0, 1, 0
    
    return pd.Series({'h_statistic': h_statistic, 'p_value': p_value, 'effect_size': effect_size})

# Define the metadata for Dask
meta = pd.DataFrame(columns=['h_statistic', 'p_value', 'effect_size']).astype({'h_statistic': 'float64', 'p_value': 'float64', 'effect_size': 'float64'})

# Apply the test to each chunk
results = df.map_partitions(lambda chunk: chunk.apply(compute_test, axis=1), meta=meta).compute()

# Perform Benjamini-Hochberg correction
_, adjusted_p_values, _, _ = multipletests(results['p_value'], method='fdr_bh')

# Add adjusted p-values to the results
results['adjusted_p_value'] = adjusted_p_values

# Calculate the number of significant results
alpha = 0.05  # Set your desired significance level
significant_count = sum(results['adjusted_p_value'] < alpha)

print(f"Number of tests performed: {len(results)}")
print(f"Number of significant results at α = {alpha}: {significant_count}")
print(f"Proportion of significant results: {significant_count / len(results):.2%}")

# Save results to file
results.to_csv('kruskal_wallis_results_abundance_matrix.csv', index=False)

# Display summary statistics
print("\nSummary Statistics:")
print(results.describe())

# Display top significant results
print("\nTop 10 significant unitigs:")
print(results.sort_values('adjusted_p_value').head(10))

# If you're using Dask distributed, uncomment the following line:
# client.close()

In [ ]:
results[results["adjusted_p_value"]<0.05].describe()

# PCA pa 

In [2]:
import dask.dataframe as dd
import numpy as np
from sklearn.decomposition import PCA
import pandas as pd
import time

print("Starting PCA process...")

# Read the CSV file, excluding the first column (Unitigs_id)
print("Reading CSV file...")
start_time = time.time()
df = dd.read_csv("../unitig_matrix_pa/unitigs.filter.02.csv", 
                 header=0, 
                 usecols=lambda x: x != 'Unitigs_id',
                 assume_missing=True, 
                 dtype='float32',
                 blocksize="1GB")
print(f"CSV file read in {time.time() - start_time:.2f} seconds.")

# Store the column names (sample names)
sample_names = df.columns

# Initialize PCA
pca = PCA(n_components=2)

# Fit and transform PCA model
print("Fitting and transforming PCA model...")
start_time = time.time()
pca_result = pca.fit_transform(df.compute())
print(f"PCA model fitted and data transformed in {time.time() - start_time:.2f} seconds.")

# Create a DataFrame with the PCA results
pca_df = pd.DataFrame(pca_result, columns=['PC1', 'PC2'])

# Get the explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_

# Save the PCA result
print("Saving PCA results...")
start_time = time.time()
pca_df.to_csv('../pca/pca_result_unitigs_pa.csv', index=False)
print(f"PCA results saved in {time.time() - start_time:.2f} seconds.")

# Save the explained variance ratio
np.savetxt('../pca/explained_variance_ratio_unitigs_pa.csv', explained_variance_ratio, delimiter=',')

# Save the component loadings with sample names as index
component_loadings = pd.DataFrame(pca.components_.T, index=sample_names, columns=['PC1', 'PC2'])
component_loadings.to_csv('../pca/component_loadings_unitigs_pa.csv')

print("PCA completed.")
print(f"Explained variance ratio: {explained_variance_ratio}")
print("Results saved to 'pca_result_unitigs_pa.csv', '../pca/explained_variance_ratio_unitigs_pa.csv', and '../pca/component_loadings_unitigs_pa.csv'")

Starting PCA process...
Reading CSV file...
CSV file read in 0.01 seconds.
Fitting and transforming PCA model...
PCA model fitted and data transformed in 34.49 seconds.
Saving PCA results...
PCA results saved in 42.89 seconds.
PCA completed.
Explained variance ratio: [0.18547074 0.02983274]
Results saved to 'pca_result_unitigs_pa.csv', 'explained_variance_ratio_unitigs_pa.csv', and 'component_loadings_unitigs_pa.csv'


# iPCA on abundance

In [3]:
import dask.dataframe as dd
import numpy as np
from sklearn.decomposition import PCA
import pandas as pd
import time

print("Starting PCA process...")

# Read the CSV file, excluding the first column (Unitigs_id)
print("Reading CSV file...")
start_time = time.time()
df = dd.read_csv("../unitig_matrix_pa/unitigs.filter.02.csv", 
                 header=0, 
                 usecols=lambda x: x != 'Unitigs_id',
                 assume_missing=True, 
                 dtype='float32',
                 blocksize="1GB")
print(f"CSV file read in {time.time() - start_time:.2f} seconds.")

# Store the sample names (column names)
sample_names = df.columns.tolist()

# Transpose the dataframe
print("Transposing the dataframe...")
start_time = time.time()
df_transposed = df.compute().transpose()
print(f"Dataframe transposed in {time.time() - start_time:.2f} seconds.")

# Initialize PCA
pca = PCA(n_components=2)

# Fit and transform PCA model
print("Fitting and transforming PCA model...")
start_time = time.time()
pca_result = pca.fit_transform(df_transposed)
print(f"PCA model fitted and data transformed in {time.time() - start_time:.2f} seconds.")

# Create a DataFrame with the PCA results
pca_df = pd.DataFrame(pca_result, index=sample_names, columns=['PC1', 'PC2'])

# Get the explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_

# Save the PCA result
print("Saving PCA results...")
start_time = time.time()
pca_df.to_csv('../pca/pca_resultunitigs_pa.csv')
print(f"PCA results saved in {time.time() - start_time:.2f} seconds.")

# Save the explained variance ratio
np.savetxt('../pca/explained_variance_ratio_unitigs_pa.csv', explained_variance_ratio, delimiter=',')

# Save the component loadings
component_loadings = pd.DataFrame(pca.components_.T, columns=['PC1', 'PC2'])
component_loadings.to_csv('component_loadings_unitigs_pa.csv', index=False)

print("PCA completed.")
print(f"Explained variance ratio: {explained_variance_ratio}")
print("Results saved to '../pca/pca_resultunitigs_pa.csv', '../pca/explained_variance_ratio_unitigs_pa.csv', and '../pca/component_loadings_unitigs_pa.csv'")

Starting PCA process...
Reading CSV file...
CSV file read in 0.01 seconds.
Transposing the dataframe...


2024-10-18 12:10:55,020 - distributed.worker.memory - WARNING - Worker is at 82% memory usage. Pausing worker.  Process memory: 15.32 GiB -- Worker memory limit: 18.63 GiB
2024-10-18 12:10:58,334 - distributed.worker.memory - WARNING - Worker is at 59% memory usage. Resuming worker. Process memory: 11.16 GiB -- Worker memory limit: 18.63 GiB


Dataframe transposed in 35.27 seconds.
Fitting and transforming PCA model...
PCA model fitted and data transformed in 73.79 seconds.
Saving PCA results...
PCA results saved in 0.17 seconds.
PCA completed.
Explained variance ratio: [0.06312743 0.03437123]
Results saved to 'pca_result_unitigs_pa.csv', 'explained_variance_ratio_unitigs_pa.csv', and 'component_loadings_unitigs_pa.csv'


# PCA on significant unitigs pa

In [24]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import time

print("Starting PCA process...")

# Read the CSV file
print("Reading CSV file...")
start_time = time.time()
df = pd.read_csv("pa.unitigs.filter.02.significant.unitigs.kruskall.csv", 
                 header=None, 
                 index_col=0,
                 dtype='float32')
print(f"CSV file read in {time.time() - start_time:.2f} seconds.")

# Rename the index
df.index.name = 'Unitigs_id'

# Read sample names from file
with open('/pasteur/appa/scratch/cduitama/Damselflies/metadata/fof_filtered_pa.txt', 'r') as file:
    sample_names = [line.strip().split('/')[-1].split('.')[0] for line in file if line.strip()]

# Set column names
df.columns = sample_names

# Initialize PCA
n_components = 2  # You can change this to the number of components you want
pca = PCA(n_components=n_components)

# Fit and transform PCA model
print("Fitting and transforming PCA model...")
start_time = time.time()
pca_result = pca.fit_transform(df.T).T  # Note the transpose operations
print(f"PCA model fitted and data transformed in {time.time() - start_time:.2f} seconds.")

# Create a DataFrame with the PCA results
pca_df = pd.DataFrame(pca_result, columns=df.columns, index=[f'PC{i+1}' for i in range(n_components)])

# Get the explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_

# Save the PCA result
print("Saving PCA results...")
start_time = time.time()
pca_df.to_csv('../pca/pca_result_unitigs_pa_filter.02_kruskal.csv')
print(f"PCA results saved in {time.time() - start_time:.2f} seconds.")

# Save the explained variance ratio
np.savetxt('../pca/explained_variance_ratio_unitigs_pa_filter.02_kruskal.csv', explained_variance_ratio, delimiter=',')

# Save the component loadings
component_loadings = pd.DataFrame(pca.components_, columns=df.index, index=[f'PC{i+1}' for i in range(n_components)])
component_loadings.to_csv('../pca/component_loadings_unitigs_pa_filter.02_kruskal.csv')

print("PCA completed.")
print(f"Explained variance ratio: {..}")
print("Results saved to:")
print("'../pca/pca_result_unitigs_pa_filter.02_kruskal.csv'")
print("'../pca/explained_variance_ratio_unitigs_pa_filter.02_kruskal.csv'")
print("'../pca/component_loadings_unitigs_pa_filter.02_kruskal.csv'")

Starting PCA process...
Reading CSV file...
CSV file read in 0.08 seconds.
Fitting and transforming PCA model...
PCA model fitted and data transformed in 0.63 seconds.
Saving PCA results...
PCA results saved in 0.03 seconds.
PCA completed.
Explained variance ratio: [0.06414654 0.03461753]
Results saved to:
'pca_result_unitigs_pa_filter.02_kruskal.csv'
'explained_variance_ratio_unitigs_pa_filter.02_kruskal.csv'
'component_loadings_unitigs_pa_filter.02_kruskal.csv'


# PCA on significant unitigs for abundance

In [4]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import time

print("Starting PCA process...")

# Read the CSV file
print("Reading CSV file...")
start_time = time.time()
df = pd.read_csv("abundance.unitigs.filter.02.significant.unitigs.kruskall.csv", 
                 sep='\s+',
                 header=None,
                 index_col=0)
print(f"CSV file read in {time.time() - start_time:.2f} seconds.")

# Set the index name
df.index.name = 'Unitigs_id'

# Read sample names from file
with open('/pasteur/appa/scratch/cduitama/Damselflies/metadata/fof_filtered.txt', 'r') as file:
    sample_names = [line.strip().split('/')[-1].split('.')[0] for line in file if line.strip()]

# Set column names
df.columns = sample_names

# Initialize PCA
n_components = 2  # You can change this to the number of components you want
pca = PCA(n_components=n_components)

# Fit and transform PCA model
print("Fitting and transforming PCA model...")
start_time = time.time()
pca_result = pca.fit_transform(df.T)
print(f"PCA model fitted and data transformed in {time.time() - start_time:.2f} seconds.")

# Create a DataFrame with the PCA results
pca_df = pd.DataFrame(pca_result, columns=[f'PC{i+1}' for i in range(n_components)], index=df.columns)

# Get the explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_

# Save the PCA result
print("Saving PCA results...")
start_time = time.time()
pca_df.to_csv('../pca/pca_result_unitigs_abundance_filter.02_kruskal.csv')
print(f"PCA results saved in {time.time() - start_time:.2f} seconds.")

# Save the explained variance ratio
np.savetxt('../pca/explained_variance_ratio_unitigs_abundance_filter.02_kruskal.csv', explained_variance_ratio, delimiter=',')

# Save the component loadings
component_loadings = pd.DataFrame(pca.components_, columns=df.index, index=[f'PC{i+1}' for i in range(n_components)])
component_loadings.to_csv('../pca/component_loadings_unitigs_abundance_filter.02_kruskal.csv')

print("PCA completed.")
print(f"Explained variance ratio: {explained_variance_ratio}")
print("Results saved to:")
print("'pca_result_unitigs_abundance_filter.02_kruskal.csv'")
print("'../pca/explained_variance_ratio_unitigs_abundance_filter.02_kruskal.csv'")
print("'../pca/component_loadings_unitigs_abundance_filter.02_kruskal.csv'")

Starting PCA process...
Reading CSV file...
CSV file read in 0.00 seconds.
Fitting and transforming PCA model...
PCA model fitted and data transformed in 0.02 seconds.
Saving PCA results...
PCA results saved in 0.02 seconds.
PCA completed.
Explained variance ratio: [0.05265512 0.04248332]
Results saved to:
'pca_result_unitigs_abundance_filter.02_kruskal.csv'
'explained_variance_ratio_unitigs_abundance_filter.02_kruskal.csv'
'component_loadings_unitigs_abundance_filter.02_kruskal.csv'
